In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # Inventory Data Processing Pipeline

import pandas as pd
import numpy as np
import json
from datetime import datetime

# Sample inventory data
data = {
    "sku": ["E100", "E200", "H300", "B400", "G500", "G510", "B410", "H305"],
    "name": ["Phone", "Headphones", "Vacuum", "Serum", "Rice", "Pasta", "Moisturizer", "Blender"],
    "category": ["Electronics", "Electronics", "Home", "Beauty", "Grocery", "Grocery", "Beauty", "Home"],
    "price_usd": [699.0, 129.0, 189.0, 42.0, 18.5, 2.2, 35.0, 99.0],
    "on_hand": [42, 12, 55, 8, 600, 1200, 19, 24],
    "incoming": [10, 0, 5, 0, 0, 0, 25, 8],
    "sold_ytd": [520, 800, 110, 900, 3400, 4100, 650, 200],
    "return_rate_pct": [4.5, 6.1, 2.0, 12.0, 1.0, 0.5, 33.0, 3.0]
}

df = pd.DataFrame(data)
df["inventory_value_usd"] = (df["on_hand"] * df["price_usd"]).round(2)

def validate_and_export_inventory():
    # ---------- Helpers ----------
    def issue(check, severity, message, idxs):
        return {"check": check, "severity": severity, "message": message, "affected_rows": [int(i) for i in idxs]}

    def check_for_negative_counts():
        issues = []
        for i, r in df.iterrows():
            neg_cols = [c for c in ["on_hand", "incoming", "sold_ytd"] if r[c] < 0]
            if neg_cols:
                issues.append(issue("negative_counts", "critical", f"Negative values in {', '.join(neg_cols)}", [i]))
        return issues

    def check_missing_required():
        issues = []
        required = ["sku", "name", "category", "on_hand"]
        for col in required:
            idxs = df[df[col].isnull()].index.tolist()
            if idxs:
                issues.append(issue("missing_required", "critical", f"Missing values in '{col}'", idxs))
        return issues

    def check_category_validity():
        valid = ["Electronics", "Home", "Beauty", "Grocery"]
        idxs = df[~df["category"].isin(valid)].index.tolist()
        return [issue("invalid_category", "critical", f"Invalid category (must be {', '.join(valid)})", idxs)] if idxs else []

    def check_duplicate_skus():
        dups = df[df.duplicated(subset=["sku"], keep=False)]
        return [issue("duplicate_skus", "critical", "Duplicate SKU(s) found", dups.index.tolist())] if not dups.empty else []

    def check_high_return_rate():
        idxs = df[df["return_rate_pct"] > 30].index.tolist()
        return [issue("high_return_rate", "warning", "Return rate > 30%", idxs)] if idxs else []

    def check_low_stock():
        idxs = df[(df["on_hand"] < 20) & (df["incoming"] == 0)].index.tolist()
        return [issue("low_stock", "warning", "Low stock and no incoming", idxs)] if idxs else []

    def check_suspicious_pricing():
        issues = []
        for cat, g in df.groupby("category"):
            if g["price_usd"].nunique() == 1 and len(g) > 1:
                issues.append(issue("suspicious_pricing", "warning", f"Identical price within category '{cat}'", g.index.tolist()))
        return issues

    def format_currency(value):
        return f"${value:,.2f}"

    def calculate_restock_tier(on_hand, incoming):
        if on_hand < 10 and incoming == 0:
            return "Immediate"
        if on_hand < 25:
            return "Soon"
        if on_hand < 50:
            return "Monitor"
        return "Healthy"

    # ---------- Run Validations ----------
    all_issues = []
    all_issues += check_for_negative_counts()
    all_issues += check_missing_required()
    all_issues += check_category_validity()
    all_issues += check_duplicate_skus()
    all_issues += check_high_return_rate()
    all_issues += check_low_stock()
    all_issues += check_suspicious_pricing()

    validation_passed = len([i for i in all_issues if i["severity"] == "critical"]) == 0

    # ---------- Exports (after validation) ----------
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # JSON summary
    total_value = float(round(df["inventory_value_usd"].sum(), 2))
    category_snapshot = {}
    for cat, g in df.groupby("category"):
        category_snapshot[cat] = {
            "item_count": int(len(g)),
            "sum_on_hand": int(g["on_hand"].sum()),
            "avg_price_usd": float(round(g["price_usd"].mean(), 2)),
        }
    at_risk = df.sort_values("return_rate_pct", ascending=False).head(5)["sku"].tolist()

    json_summary = {
        "metadata": {
            "export_date": ts,
            "total_skus": int(len(df)),
            "total_inventory_value_usd": float(round(total_value, 2))
        },
        "category_snapshot": category_snapshot,
        "at_risk_items": at_risk
    }

    # CSV data
    csv_df = df.copy()
    csv_df["restock_tier"] = [calculate_restock_tier(r.on_hand, r.incoming) for _, r in csv_df.iterrows()]
    csv_df = csv_df.sort_values("sku")
    csv_data = csv_df.to_dict("records")

    # TXT report
    txt = []
    txt.append("Inventory Report")
    txt.append(f"Generated: {ts}\n")
    txt.append("Summary:")
    txt.append(f"- Total SKUs: {len(df)}")
    txt.append(f"- Inventory Value: {format_currency(total_value)}")
    txt.append(f"- Avg Price: {format_currency(df['price_usd'].mean())}\n")

    txt.append("Category Breakdown:")
    for cat in sorted(df["category"].unique()):
        g = df[df["category"] == cat]
        txt.append(f"{cat}: count={len(g)}, on_hand={int(g['on_hand'].sum())}, avg_price={format_currency(g['price_usd'].mean())}")
    txt.append("\nAt-Risk Items (highest return rate):")
    for i, sku in enumerate(at_risk, 1):
        rr = df.loc[df["sku"] == sku, "return_rate_pct"].values[0]
        txt.append(f"{i}. {sku} - {rr:.2f}%")

    txt.append(f"\nValidation Status: {'PASSED' if validation_passed else 'FAILED'}")
    txt.append(f"Issues Found: {len(all_issues)}")
    txt_report = "\n".join(txt)

    exports_generated = []
    try:
        with open("inventory_summary.json", "w") as f:
            json.dump(json_summary, f, indent=2)
        exports_generated.append("inventory_summary.json")
    except Exception as e:
        print("JSON write error:", e)

    try:
        csv_df.to_csv("inventory_detailed.csv", index=False)
        exports_generated.append("inventory_detailed.csv")
    except Exception as e:
        print("CSV write error:", e)

    try:
        with open("inventory_report.txt", "w") as f:
            f.write(txt_report)
        exports_generated.append("inventory_report.txt")
    except Exception as e:
        print("TXT write error:", e)

    return {
        "validation_passed": validation_passed,
        "validation_issues": all_issues,
        "exports_generated": exports_generated,
        "export_data": {
            "json_summary": json_summary,
            "csv_data": csv_data,
            "txt_report": txt_report
        }
    }

# Execute
result = validate_and_export_inventory()
